In [1]:
import sys
sys.path.append('..')

In [2]:
import json
import random
import numpy as np
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

from transformers import BertModel, BertTokenizer, BertConfig, AdamW, get_linear_schedule_with_warmup
from data_utils import (
    load_dataset, 
    get_examples_from_dialogues, 
    convert_state_dict, 
    DSTInputExample, 
    OpenVocabDSTFeature, 
    DSTPreprocessor, 
    WOSDataset)
    
from inference import inference
from evaluation import _evaluation

## Data loading

In [3]:
train_data_file = "/opt/ml/repo/taepd/input/data/train_dataset/train_dials.json"
slot_meta = json.load(open("/opt/ml/repo/taepd/input/data/train_dataset/slot_meta.json"))
ontology = json.load(open("/opt/ml/repo/taepd/input/data/train_dataset/ontology.json"))
train_data, dev_data, dev_labels = load_dataset(train_data_file)

# dialogue_level=False : SUMBT와 다르게 dialogue context level로 input하므로
train_examples = get_examples_from_dialogues(train_data,
                                             user_first=False,
                                             dialogue_level=False)
dev_examples = get_examples_from_dialogues(dev_data,
                                           user_first=False,
                                           dialogue_level=False)

100%|██████████| 699/699 [00:00<00:00, 16390.69it/s]


In [4]:
print(len(train_examples))
print(len(dev_examples))

46217
5028


In [5]:
print(train_examples[2])

DSTInputExample(guid='polished-poetry-0057:관광_9-2', context_turns=['', '쇼핑을 하려는데 서울 서쪽에 있을까요?', '서울 서쪽에 쇼핑이 가능한 곳이라면 노량진 수산물 도매시장이 있습니다.', '오 네 거기 주소 좀 알려주세요.'], current_turn=['노량진 수산물 도매시장의 주소는 서울 동작구 93806입니다.', '알려주시는김에 연락처랑 평점도 좀 알려주세요.'], label=['관광-종류-쇼핑', '관광-지역-서울 서쪽', '관광-이름-노량진 수산물 도매시장'])


## TRADE Preprocessor 

기존의 GRU 기반의 인코더를 BERT-based Encoder로 바꿀 준비를 합시다.

1. 현재 `_convert_example_to_feature`에서는 `max_seq_length`를 핸들하고 있지 않습니다. `input_id`와 `segment_id`가 `max_seq_length`를 넘어가면 좌측부터 truncate시키는 코드를 삽입하세요.

2. `recover_state`를 구현해 보세요.

In [49]:
class TRADEPreprocessor(DSTPreprocessor):
    def __init__(
        self,
        slot_meta,
        src_tokenizer,
        trg_tokenizer=None,
        ontology=None,
        max_seq_length=512,
    ):
        self.slot_meta = slot_meta
        self.src_tokenizer = src_tokenizer
        self.trg_tokenizer = trg_tokenizer if trg_tokenizer else src_tokenizer
        self.ontology = ontology     
        self.gating2id = {"none": 0, "dontcare": 1, "ptr": 2}     
        self.id2gating = {v: k for k, v in self.gating2id.items()}
        self.max_seq_length = max_seq_length
#         self.input_arr = []
    def _convert_example_to_feature(self, example):
        dialogue_context = " [SEP] ".join(example.context_turns + example.current_turn)
        input_id = self.src_tokenizer.encode(dialogue_context, add_special_tokens=False)
#         self.input_arr.append(len(input_id))
        max_length = self.max_seq_length - 2
        
        # naive approach 그냥 좌측에서 max_len-input_id만큼 slicing
#         gap = len(input_id) - max_length
#         if gap > 0:
#             input_id = input_id[gap:]

        if len(input_id) > max_length:
            gap = len(input_id) - max_length
            input_id = input_id[gap:]
        # padding 처리는 collate_fn에서 처리

        input_id = (
            [self.src_tokenizer.cls_token_id]
            + input_id
            + [self.src_tokenizer.sep_token_id]
        )
        segment_id = [0] * len(input_id)  # bert기반이 되었을 땐 segment 구분이 필요할 수 있음

        target_ids = []
        gating_id = []
        if not example.label:
            example.label = []

        state = convert_state_dict(example.label)
        for slot in self.slot_meta:
            value = state.get(slot, "none")
            target_id = self.trg_tokenizer.encode(value, add_special_tokens=False) + [
                self.trg_tokenizer.sep_token_id
            ]  # [sep]를 추가하여, AutoRegressive시, EOS token으로 사용
            target_ids.append(target_id)
            gating_id.append(self.gating2id.get(value, self.gating2id["ptr"]))  # slot meta와 같은 len을 가진 0,1,2의 배열
            
        target_ids = self.pad_ids(target_ids, self.trg_tokenizer.pad_token_id)  # collate_fn에서 처리해주는 것 같은데?
        
        return OpenVocabDSTFeature(
            example.guid, input_id, segment_id, gating_id, target_ids
        )

    def convert_examples_to_features(self, examples):
        return list(map(self._convert_example_to_feature, examples))

    def recover_state(self, gate_list, gen_list):
        # problem 2.
        # Your code here!
        # raise Exception('TRADE의 아웃풋을 prediction form으로 바꾸는 코드를 작성하세요!')
        assert len(gate_list) == len(self.slot_meta)
        assert len(gen_list) == len(self.slot_meta)

        recovered = []
        for slot, gate, value in zip(self.slot_meta, gate_list, gen_list):
            if self.id2gating[gate] == "none":
                continue

            if self.id2gating[gate] == "dontcare":
                recovered.append("%s-%s" % (slot, "dontcare"))
                continue

            token_id_list = []
            for id_ in value:
                if id_ in self.trg_tokenizer.all_special_ids:
                    break

                token_id_list.append(id_)
            value = self.trg_tokenizer.decode(token_id_list, skip_special_tokens=True)

            if value == "none":
                continue

            recovered.append("%s-%s" % (slot, value))
        return recovered

    def collate_fn(self, batch):
        guids = [b.guid for b in batch]
        input_ids = torch.LongTensor(
            self.pad_ids([b.input_id for b in batch], self.src_tokenizer.pad_token_id)
        )
        segment_ids = torch.LongTensor(
            self.pad_ids([b.segment_id for b in batch], self.src_tokenizer.pad_token_id)
        )
        input_masks = input_ids.ne(self.src_tokenizer.pad_token_id)

        gating_ids = torch.LongTensor([b.gating_id for b in batch])
        target_ids = self.pad_id_of_matrix(
            [torch.LongTensor(b.target_ids) for b in batch],
            self.trg_tokenizer.pad_token_id,
        )
        return input_ids, segment_ids, input_masks, gating_ids, target_ids, guids

## Convert_Examples_to_Features 

In [50]:
tokenizer = BertTokenizer.from_pretrained('dsksd/bert-ko-small-minimal')
processor = TRADEPreprocessor(slot_meta, tokenizer, max_seq_length=512)

train_features = processor.convert_examples_to_features(train_examples)
dev_features = processor.convert_examples_to_features(dev_examples)

chk  [SEP] 쇼핑을 하려는데 서울 서쪽에 있을까요?
chk  [SEP] 쇼핑을 하려는데 서울 서쪽에 있을까요? [SEP] 서울 서쪽에 쇼핑이 가능한 곳이라면 노량진 수산물 도매시장이 있습니다. [SEP] 오 네 거기 주소 좀 알려주세요.
chk  [SEP] 쇼핑을 하려는데 서울 서쪽에 있을까요? [SEP] 서울 서쪽에 쇼핑이 가능한 곳이라면 노량진 수산물 도매시장이 있습니다. [SEP] 오 네 거기 주소 좀 알려주세요. [SEP] 노량진 수산물 도매시장의 주소는 서울 동작구 93806입니다. [SEP] 알려주시는김에 연락처랑 평점도 좀 알려주세요.
chk  [SEP] 쇼핑을 하려는데 서울 서쪽에 있을까요? [SEP] 서울 서쪽에 쇼핑이 가능한 곳이라면 노량진 수산물 도매시장이 있습니다. [SEP] 오 네 거기 주소 좀 알려주세요. [SEP] 노량진 수산물 도매시장의 주소는 서울 동작구 93806입니다. [SEP] 알려주시는김에 연락처랑 평점도 좀 알려주세요. [SEP] 그럼. 연락처는 6182006591이고 평점은 4점입니다. [SEP] 와 감사합니다.
chk  [SEP] 안녕하세요? 적당한 가격대의 양식당을 친구들이랑 방문하려고 하는데요. 어차피 지하철을 이용할거여서 서울 안에만 위치한 곳으로 찾아봐주시겠어요?
chk  [SEP] 안녕하세요? 적당한 가격대의 양식당을 친구들이랑 방문하려고 하는데요. 어차피 지하철을 이용할거여서 서울 안에만 위치한 곳으로 찾아봐주시겠어요? [SEP] 안녕하세요? 서울 광희동에 위치한 평점 4점의 어차피자라는 곳은 어떠신가요? [SEP] 피자집인거죠? 괜찮네요. 저 여기로 금요일 3시에 4명으로 예약 부탁드립니다.
chk  [SEP] 안녕하세요? 적당한 가격대의 양식당을 친구들이랑 방문하려고 하는데요. 어차피 지하철을 이용할거여서 서울 안에만 위치한 곳으로 찾아봐주시겠어요? [SEP] 안녕하세요? 서울 광희동에 위치한 평점 4점의 어차피자라는 곳은 어떠신가요? [SEP] 피자집인거죠? 괜찮네요. 저 여기로 금요일 3시에 

Token indices sequence length is longer than the specified maximum sequence length for this model (537 > 512). Running this sequence through the model will result in indexing errors


 [SEP] 안녕하세요. 서울 남쪽에 저렴한 가격대의 숙소가 있나요? [SEP] 안녕하세요. 네, 한번 확인해보겠습니다. 원하는 종류는 어떻게 되시나요? [SEP] 음.. 저는 종류는 상관은 없는데 스파를 좋아해서요. 스파가 꼭 있는 곳으로 찾아주세요. [SEP] 네. 잘 알겠습니다. 고객님, 죄송하지만 해당 조건으로 검색되는 숙소가 0건입니다. 혹시 가격대 변경 가능하실까요? [SEP] 아 이 가격대로는 제가 요청드린 숙소를 찾기에 터무니 없었나봐요. 음.. 그러면 혹시 적당한 가격대로 봐주실 수 있으세요?
chk  [SEP] 안녕하세요. 서울 남쪽에 저렴한 가격대의 숙소가 있나요? [SEP] 안녕하세요. 네, 한번 확인해보겠습니다. 원하는 종류는 어떻게 되시나요? [SEP] 음.. 저는 종류는 상관은 없는데 스파를 좋아해서요. 스파가 꼭 있는 곳으로 찾아주세요. [SEP] 네. 잘 알겠습니다. 고객님, 죄송하지만 해당 조건으로 검색되는 숙소가 0건입니다. 혹시 가격대 변경 가능하실까요? [SEP] 아 이 가격대로는 제가 요청드린 숙소를 찾기에 터무니 없었나봐요. 음.. 그러면 혹시 적당한 가격대로 봐주실 수 있으세요? [SEP] 바로 확인 도와드릴게요. 서울 강남구에 위치한 2곳이 확인 되는데요, 포레스트 호텔과 패밀리 호텔이 있습니다. 평점도 거의 비슷해서 둘다 비슷한 조건이라고 보시면 좋을 것 같아요. [SEP] 뭔가 포레스트 호텔이 좀 더 세련된 호텔일 것 같아요. 저 여기로 2명 좀 예약해주세요. 일요일부터 3일간 머무를 예정입니다.
chk  [SEP] 안녕하세요. 서울 남쪽에 저렴한 가격대의 숙소가 있나요? [SEP] 안녕하세요. 네, 한번 확인해보겠습니다. 원하는 종류는 어떻게 되시나요? [SEP] 음.. 저는 종류는 상관은 없는데 스파를 좋아해서요. 스파가 꼭 있는 곳으로 찾아주세요. [SEP] 네. 잘 알겠습니다. 고객님, 죄송하지만 해당 조건으로 검색되는 숙소가 0건입니다. 혹시 가격대 변경 가능하실까요? [SEP] 아 이 가격대로는 제가 

KeyboardInterrupt: 

In [9]:
print(len(train_features))
print(len(dev_features))

46217
5028


In [16]:
print(np.mean(np.array(processor.input_arr)))
print(np.max(np.array(processor.input_arr)))
train_features[1].target_ids

171.6817835886428
934


[[21832, 11764, 3, 0, 0],
 [21832, 11764, 3, 0, 0],
 [21832, 11764, 3, 0, 0],
 [21832, 11764, 3, 0, 0],
 [21832, 11764, 3, 0, 0],
 [28060, 16301, 15550, 12178, 3],
 [7596, 3, 0, 0, 0],
 [21832, 11764, 3, 0, 0],
 [6265, 10806, 3, 0, 0],
 [21832, 11764, 3, 0, 0],
 [21832, 11764, 3, 0, 0],
 [21832, 11764, 3, 0, 0],
 [21832, 11764, 3, 0, 0],
 [21832, 11764, 3, 0, 0],
 [21832, 11764, 3, 0, 0],
 [21832, 11764, 3, 0, 0],
 [21832, 11764, 3, 0, 0],
 [21832, 11764, 3, 0, 0],
 [21832, 11764, 3, 0, 0],
 [21832, 11764, 3, 0, 0],
 [21832, 11764, 3, 0, 0],
 [21832, 11764, 3, 0, 0],
 [21832, 11764, 3, 0, 0],
 [21832, 11764, 3, 0, 0],
 [21832, 11764, 3, 0, 0],
 [21832, 11764, 3, 0, 0],
 [21832, 11764, 3, 0, 0],
 [21832, 11764, 3, 0, 0],
 [21832, 11764, 3, 0, 0],
 [21832, 11764, 3, 0, 0],
 [21832, 11764, 3, 0, 0],
 [21832, 11764, 3, 0, 0],
 [21832, 11764, 3, 0, 0],
 [21832, 11764, 3, 0, 0],
 [21832, 11764, 3, 0, 0],
 [21832, 11764, 3, 0, 0],
 [21832, 11764, 3, 0, 0],
 [21832, 11764, 3, 0, 0],
 [21832, 1

# Model 

In [17]:
class TRADE(nn.Module):
    def __init__(self, config, slot_vocab, slot_meta, pad_idx=0):
        super(TRADE, self).__init__()
        self.slot_meta = slot_meta
        
        self.encoder = GRUEncoder(
            config.vocab_size,
            config.hidden_size,
            1,
            config.hidden_dropout_prob,
            config.proj_dim,
            pad_idx,
        )

        self.decoder = SlotGenerator(
            config.vocab_size,
            config.hidden_size,
            config.hidden_dropout_prob,
            config.n_gate,
            None,
            pad_idx,
        )
        
        
        # init for only subword embedding
        # 강좌에만 있는 코드
        if config.model_name_or_path:
            bert = BertModel.from_pretrained(config.model_name_or_path)
        else:
            bert = BertModel(config)
        self.encoder.embed.weight = bert.embeddings.word_embeddings.weight
        # 강좌에만 있는 코드
        
        self.decoder.set_slot_idx(slot_vocab)
        self.tie_weight()  

    # encoder와 decoder의 embedding matrix sharing        
    def tie_weight(self):
        self.decoder.embed.weight = self.encoder.embed.weight
        if self.decoder.proj_layer:
            self.decoder.proj_layer.weight = self.encoder.proj_layer.weight

    def forward(self, 
                input_ids,  # p^history를 vocal size로 변환하기 위해 input_ids받음      
                token_type_ids, 
                attention_mask=None, 
                max_len=10, # maximum length를 target_id에서 받고, inference 때는 고정된 constant를 받음
                teacher=None): # teacher forcing을 위한 GT Sequence

        encoder_outputs, pooled_output = self.encoder(input_ids=input_ids)
        all_point_outputs, all_gate_outputs = self.decoder(  
            input_ids, encoder_outputs, pooled_output.unsqueeze(0), attention_mask, max_len, teacher  
        )

        return all_point_outputs, all_gate_outputs
    

class GRUEncoder(nn.Module):
    def __init__(self, vocab_size, d_model, n_layer, dropout, proj_dim=None, pad_idx=0):
        super(GRUEncoder, self).__init__()
        self.pad_idx = pad_idx
        self.embed = nn.Embedding(vocab_size, d_model, padding_idx=pad_idx)
        if proj_dim:
            self.proj_layer = nn.Linear(d_model, proj_dim, bias=False)
        else:
            self.proj_layer = None

        self.d_model = proj_dim if proj_dim else d_model
        self.gru = nn.GRU(
            self.d_model,
            self.d_model,
            n_layer,
            dropout=dropout,
            batch_first=True,
            bidirectional=True,  # bidirectianl GRU
        )
        self.dropout = nn.Dropout(dropout)

    def forward(self, input_ids):
        mask = input_ids.eq(self.pad_idx).unsqueeze(-1)  # pad를 masking하기 위해
        x = self.embed(input_ids)
        if self.proj_layer:
            x = self.proj_layer(x)
        x = self.dropout(x)
        o, h = self.gru(x)
        o = o.masked_fill(mask, 0.0)
        output = o[:, :, : self.d_model] + o[:, :, self.d_model :]  # output summation
        hidden = h[0] + h[1]  # n_layer 고려  hidden vector, decoder의 initial hidden state 
        return output, hidden
    
    
class SlotGenerator(nn.Module):
    def __init__(
        self, vocab_size, hidden_size, dropout, n_gate, proj_dim=None, pad_idx=0
    ):
        super(SlotGenerator, self).__init__()
        self.pad_idx = pad_idx
        self.vocab_size = vocab_size
        self.embed = nn.Embedding(
            vocab_size, hidden_size, padding_idx=pad_idx
        )  # shared with encoder

        if proj_dim:
            self.proj_layer = nn.Linear(hidden_size, proj_dim, bias=False)
        else:
            self.proj_layer = None
        self.hidden_size = proj_dim if proj_dim else hidden_size

        self.gru = nn.GRU(
            self.hidden_size, self.hidden_size, 1, dropout=dropout, batch_first=True
        )  # unidirectional GRU
        self.n_gate = n_gate
        self.dropout = nn.Dropout(dropout)
        self.w_gen = nn.Linear(self.hidden_size * 3, 1)  # p_gen을 만들 때 3가지를 concat하므로 *3
        self.sigmoid = nn.Sigmoid()
        self.w_gate = nn.Linear(self.hidden_size, n_gate)

    def set_slot_idx(self, slot_vocab_idx):
        whole = []
        max_length = max(map(len, slot_vocab_idx))  # tokenized slot_meta 길이 최댓값: ex)4
        for idx in slot_vocab_idx:
            if len(idx) < max_length:
                gap = max_length - len(idx)
                idx.extend([self.pad_idx] * gap)
            whole.append(idx)
        self.slot_embed_idx = whole  # ex) [[6728, 6479, 1, 1],..]

    def embedding(self, x):
        x = self.embed(x)
        if self.proj_layer:
            x = self.proj_layer(x)
        return x

    def forward(
        self, input_ids, encoder_output, hidden, input_masks, max_len, teacher=None
    ):
        input_masks = input_masks.ne(1)
        # J, slot_meta : key : [domain, slot] ex> LongTensor([1,2]) < 이 헷깔리는 주석의 의미는 뭘까
        # J,2
        batch_size = encoder_output.size(0)
        slot = torch.LongTensor(self.slot_embed_idx).to(input_ids.device)  ##
        slot_e = torch.sum(self.embedding(slot), 1)  # J,d. J * slot_vocab_idx 를 embedding을 거쳐서 sum해주어 J * hidden 이 됨
        J = slot_e.size(0)

        all_point_outputs = torch.zeros(batch_size, J, max_len, self.vocab_size).to(  # max_len: max gen len
            input_ids.device
        )  # Output Tensor (for Placeholder)
        
        # Parallel Decoding
        w = slot_e.repeat(batch_size, 1).unsqueeze(1)  # domain-slot embedding
        hidden = hidden.repeat_interleave(J, dim=1)
        encoder_output = encoder_output.repeat_interleave(J, dim=0)
        input_ids = input_ids.repeat_interleave(J, dim=0)
        input_masks = input_masks.repeat_interleave(J, dim=0)
        for k in range(max_len):
            w = self.dropout(w)
            _, hidden = self.gru(w, hidden)  # 1,B,D

            # B,T,D * B,D,1 => B,T,1  (T가 |X_t|인듯, input sequence length)
            attn_e = torch.bmm(encoder_output, hidden.permute(1, 2, 0))  # B,T,1
            attn_e = attn_e.squeeze(-1).masked_fill(input_masks, -1e9)
            attn_history = F.softmax(attn_e, -1)  # B,T

            if self.proj_layer:
                hidden_proj = torch.matmul(hidden, self.proj_layer.weight)
            else:
                hidden_proj = hidden

            # B,D * D,V => B,V
            attn_v = torch.matmul(
                hidden_proj.squeeze(0), self.embed.weight.transpose(0, 1)
            )  # B,V
            attn_vocab = F.softmax(attn_v, -1)

            # B,1,T * B,T,D => B,1,D
            context = torch.bmm(attn_history.unsqueeze(1), encoder_output)  # B,1,D
            p_gen = self.sigmoid(
                self.w_gen(torch.cat([w, hidden.transpose(0, 1), context], -1))
            )  # B,1
            p_gen = p_gen.squeeze(-1)

            p_context_ptr = torch.zeros_like(attn_vocab).to(input_ids.device)
            p_context_ptr.scatter_add_(1, input_ids, attn_history)  # copy B,V
            p_final = p_gen * attn_vocab + (1 - p_gen) * p_context_ptr  # B,V
            _, w_idx = p_final.max(-1)

            if teacher is not None:
                w = self.embedding(teacher[:, :, k]).transpose(0, 1).reshape(batch_size * J, 1, -1)
            else:
                w = self.embedding(w_idx).unsqueeze(1)  # B,1,D
            if k == 0:
                gated_logit = self.w_gate(context.squeeze(1))  # B*J,n_gate
                all_gate_outputs = gated_logit.view(batch_size, J, self.n_gate)
            all_point_outputs[:, :, k, :] = p_final.view(batch_size, J, self.vocab_size)  # B, J, k, V

        return all_point_outputs, all_gate_outputs

# 모델 및 데이터 로더 정의

In [18]:
slot_vocab = []
for slot in slot_meta:
    slot_vocab.append(
        tokenizer.encode(slot.replace('-', ' '),  # 원래는 domain, slot을 영어와 달리 동일 길이로 토큰화되지 않기에 그냥 concat
                         add_special_tokens=False)
    )
print(slot_vocab[1])

config = BertConfig.from_pretrained('dsksd/bert-ko-small-minimal')
config.model_name_or_path = 'dsksd/bert-ko-small-minimal'
config.n_gate = len(processor.gating2id)
config.proj_dim = None
model = TRADE(config, slot_vocab, slot_meta)

print(config.vocab_size)

[6728, 6295, 4199]


/opt/conda/lib/python3.7/site-packages/torch/nn/modules/rnn.py:61: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


35000


In [19]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_data = WOSDataset(train_features)
train_sampler = RandomSampler(train_data)
train_loader = DataLoader(train_data, batch_size=4, sampler=train_sampler, collate_fn=processor.collate_fn)

dev_data = WOSDataset(dev_features)
dev_sampler = SequentialSampler(dev_data)
dev_loader = DataLoader(dev_data, batch_size=8, sampler=dev_sampler, collate_fn=processor.collate_fn)

# Optimizer & Scheduler 선언

In [47]:
n_epochs = 10
no_decay = ["bias", "LayerNorm.weight"]
optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            "weight_decay": 0.01,
        },
        {
            "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
            "weight_decay": 0.0,
        },
    ]

t_total = len(train_loader) * n_epochs
optimizer = AdamW(optimizer_grouped_parameters, lr=3e-5, eps=1e-8)
scheduler = get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps=0.1, num_training_steps=t_total
)
teacher_forcing = 0.5
model.to(device)

def masked_cross_entropy_for_value(logits, target, pad_idx=0):  # logits: (B, J, k, V)
    mask = target.ne(pad_idx)
    logits_flat = logits.view(-1, logits.size(-1))  # B*J*k, V (ex) [900, 35000])
    log_probs_flat = torch.log(logits_flat)
    target_flat = target.view(-1, 1)  # 열벡터로 변환  [900, 1]
    losses_flat = -torch.gather(log_probs_flat, dim=1, index=target_flat)  # [900, 1]
    losses = losses_flat.view(*target.size())  # 행벡터로 변환 [900]
    losses = losses * mask.float()  # mask(아마도 padding?)은 제외
    loss = losses.sum() / (mask.sum().float())
    return loss

loss_fnc_1 = masked_cross_entropy_for_value  # generation
loss_fnc_2 = nn.CrossEntropyLoss()  # gating

## Train

In [48]:
for epoch in range(n_epochs):
    batch_loss = []
    model.train()
    for step, batch in enumerate(train_loader):
        input_ids, segment_ids, input_masks, gating_ids, target_ids, _ = [b.to(device) if not isinstance(b, list) else b for b in batch]
        if teacher_forcing > 0.0 and random.random() < teacher_forcing:
            tf = target_ids
        else:
            tf = None

        all_point_outputs, all_gate_outputs = model(input_ids, segment_ids, input_masks, target_ids.size(-1))  # gt - length (generation)
        loss_1 = loss_fnc_1(all_point_outputs.contiguous(), target_ids.contiguous().view(-1))
        loss_2 = loss_fnc_2(all_gate_outputs.contiguous().view(-1, 3), gating_ids.contiguous().view(-1))
        loss = loss_1 + loss_2

        batch_loss.append(loss.item())

        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
        if step % 100 == 0:            
            print('[%d/%d] [%d/%d] %f' % (epoch, n_epochs, step, len(train_loader), loss.item()))

predictions = inference(model, dev_loader, processor, device)
eval_result = _evaluation(predictions, dev_labels, slot_meta)
for k, v in eval_result.items():
    print(f"{k}: {v}")

tensor([[[21832, 11764,     3,  ...,     0,     0,     0],
         [21832, 11764,     3,  ...,     0,     0,     0],
         [21832, 11764,     3,  ...,     0,     0,     0],
         ...,
         [ 6510,     3,     0,  ...,     0,     0,     0],
         [33922,  4019, 21172,  ...,     3,     0,     0],
         [ 2181,  4887,  4127,  ...,  4756,     3,     0]],

        [[21832, 11764,     3,  ...,     0,     0,     0],
         [21832, 11764,     3,  ...,     0,     0,     0],
         [21832, 11764,     3,  ...,     0,     0,     0],
         ...,
         [21832, 11764,     3,  ...,     0,     0,     0],
         [21832, 11764,     3,  ...,     0,     0,     0],
         [21832, 11764,     3,  ...,     0,     0,     0]],

        [[21832, 11764,     3,  ...,     0,     0,     0],
         [21832, 11764,     3,  ...,     0,     0,     0],
         [21832, 11764,     3,  ...,     0,     0,     0],
         ...,
         [21832, 11764,     3,  ...,     0,     0,     0],
         [

KeyboardInterrupt: 

## Inference 

In [ ]:
eval_data = json.load(open(f"/opt/ml/input/data/eval/eval_dials.json", "r"))

eval_examples = get_examples_from_dialogues(
    eval_data, user_first=False, dialogue_level=False
)

# Extracting Featrues
eval_features = processor.convert_examples_to_features(eval_examples)
eval_data = WOSDataset(eval_features)
eval_sampler = SequentialSampler(eval_data)
eval_loader = DataLoader(
    eval_data,
    batch_size=8,
    sampler=eval_sampler,
    collate_fn=processor.collate_fn,
)

In [ ]:
predictions = inference(model, eval_loader, processor, device)

In [ ]:
json.dumps(predictions, open('predictions.csv', 'w'), indent=2, ensure_ascii=False) 